author:sukang  
time:2022-02-17  
reference:https://github.com/xiaolai-sqlai/mobilenetv3  
title:MV3-large

In [1]:
import matplotlib.pyplot as plt
from torch.cuda.amp import autocast 
from torch.cuda.amp import GradScaler
import torch
import os
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torch import optim
import torch.nn.functional as F
import numpy as np
import pandas as pd
import torch.nn as nn
import torchvision
from torchsummary import summary
import time
from torch.nn import init

In [2]:
#优化参数
#cudnn参数  会额外增加一部分显存
torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True

In [3]:
#保存训练数据和模型
data_csv_path = "D:\\OneModel\\MV3_large_optimizer_Cos.csv"   #修改此处文件名 
model_save_path = "D:\\OneModel\\MV3_large_optimizer_Cos.pkl"  #修改此处文件名

In [4]:
#设置数据集路径
train_path = "D:\\Dataset\\RAF-DB\\train"
val_path = "D:\\Dataset\\RAF-DB\\test"
#模型批次大小
batch_size = 128
resume = True
#动态学习率，学习率和循环次数增加
lr = 5e-4 
epochs = 200
D_epoch = 0 
best_acc  = 0
print("epochs:",epochs,"learning_rate:",lr,"batch_size:",batch_size)

epochs: 200 learning_rate: 0.0005 batch_size: 128


In [5]:
class hswish(nn.Module):
    def forward(self, x):
        out = x * F.relu6(x + 3, inplace=True) / 6
        return out


class hsigmoid(nn.Module):
    def forward(self, x):
        out = F.relu6(x + 3, inplace=True) / 6
        return out


class SeModule(nn.Module):
    def __init__(self, in_size, reduction=4):
        super(SeModule, self).__init__()
        self.se = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(in_size, in_size // reduction, kernel_size=1, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(in_size // reduction),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_size // reduction, in_size, kernel_size=1, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(in_size),
            hsigmoid()
        )

    def forward(self, x):
        return x * self.se(x)


class Block(nn.Module):
    '''expand + depthwise + pointwise'''
    def __init__(self, kernel_size, in_size, expand_size, out_size, nolinear, semodule, stride):
        super(Block, self).__init__()
        self.stride = stride
        self.se = semodule

        self.conv1 = nn.Conv2d(in_size, expand_size, kernel_size=1, stride=1, padding=0, bias=False)
        self.bn1 = nn.BatchNorm2d(expand_size)
        self.nolinear1 = nolinear
        self.conv2 = nn.Conv2d(expand_size, expand_size, kernel_size=kernel_size, stride=stride, padding=kernel_size//2, groups=expand_size, bias=False)
        self.bn2 = nn.BatchNorm2d(expand_size)
        self.nolinear2 = nolinear
        self.conv3 = nn.Conv2d(expand_size, out_size, kernel_size=1, stride=1, padding=0, bias=False)
        self.bn3 = nn.BatchNorm2d(out_size)

        self.shortcut = nn.Sequential()
        if stride == 1 and in_size != out_size:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_size, out_size, kernel_size=1, stride=1, padding=0, bias=False),
                nn.BatchNorm2d(out_size),
            )

    def forward(self, x):
        out = self.nolinear1(self.bn1(self.conv1(x)))
        out = self.nolinear2(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        if self.se != None:
            out = self.se(out)
        out = out + self.shortcut(x) if self.stride==1 else out
        return out


class MobileNetV3_Large(nn.Module):
    def __init__(self, num_classes=7):
        super(MobileNetV3_Large, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=2, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.hs1 = hswish()

        self.bneck = nn.Sequential(
            Block(3, 16, 16, 16, nn.ReLU(inplace=True), None, 1),
            Block(3, 16, 64, 24, nn.ReLU(inplace=True), None, 2),
            Block(3, 24, 72, 24, nn.ReLU(inplace=True), None, 1),
            Block(5, 24, 72, 40, nn.ReLU(inplace=True), SeModule(40), 2),
            Block(5, 40, 120, 40, nn.ReLU(inplace=True), SeModule(40), 1),
            Block(5, 40, 120, 40, nn.ReLU(inplace=True), SeModule(40), 1),
            Block(3, 40, 240, 80, hswish(), None, 2),
            Block(3, 80, 200, 80, hswish(), None, 1),
            Block(3, 80, 184, 80, hswish(), None, 1),
            Block(3, 80, 184, 80, hswish(), None, 1),
            Block(3, 80, 480, 112, hswish(), SeModule(112), 1),
            Block(3, 112, 672, 112, hswish(), SeModule(112), 1),
            Block(5, 112, 672, 160, hswish(), SeModule(160), 1),
            Block(5, 160, 672, 160, hswish(), SeModule(160), 2),
            Block(5, 160, 960, 160, hswish(), SeModule(160), 1),
        )


        self.conv2 = nn.Conv2d(160, 960, kernel_size=1, stride=1, padding=0, bias=False)
        self.bn2 = nn.BatchNorm2d(960)
        self.hs2 = hswish()
        self.apool = nn.AdaptiveAvgPool2d(1)
        self.linear3 = nn.Linear(960, 1280)
        self.bn3 = nn.BatchNorm1d(1280)
        self.hs3 = hswish()
        self.linear4 = nn.Linear(1280, num_classes)
        self.init_params()

    def init_params(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                init.kaiming_normal_(m.weight, mode='fan_out')
                if m.bias is not None:
                    init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                init.constant_(m.weight, 1)
                init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                init.normal_(m.weight, std=0.001)
                if m.bias is not None:
                    init.constant_(m.bias, 0)

    def forward(self, x):
        out = self.hs1(self.bn1(self.conv1(x)))
        out = self.bneck(out)
        out = self.hs2(self.bn2(self.conv2(out)))
        out = self.apool(out)
        out =  torch.flatten(out, 1)
        out = self.hs3(self.bn3(self.linear3(out)))
        out = self.linear4(out)
        return out


In [6]:
#设备选取
flag = torch.cuda.is_available()
if flag:
    print("GPU")
else:
    print("CPU")
ngpu = 1
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")
# device = torch.device("cpu")
#查看显卡名称
#torch.cuda.get_device_name()
print("divice is ", device)

#数据预处理（建议提前resize，减少每次资源的损失）放大到112x112 ，随机水平翻转
train_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225])
])

#增加不同种transform，预测集中去除随机翻转
val_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225])
])

GPU
divice is  cuda:0


In [7]:
#使用torchvision.datasets.ImageFolder读取数据集指定train和test文件夹
train_data = torchvision.datasets.ImageFolder(train_path, transform=train_transform)
#drop_last舍弃未满一个批次的数据        num_workers工作区一般设置为GPU个数的4倍
data0_train = DataLoader(train_data, batch_size=batch_size, shuffle=True, drop_last=True,num_workers=4)
print(train_data)  #输出训练集相关
val_data = torchvision.datasets.ImageFolder(val_path, transform=val_transform)
data1_val = DataLoader(val_data, batch_size=batch_size, shuffle=True,drop_last=True,num_workers=4)
print(val_data)  #输出测试集相关

print(train_data.classes)  #根据分的文件夹的名字来确定的类别
print(train_data.class_to_idx) #按顺序为这些类别定义索引为0,1...
print()
print(val_data.classes)
print(val_data.class_to_idx)
print()


Dataset ImageFolder
    Number of datapoints: 12271
    Root location: D:\Dataset\RAF-DB\train
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=None)
               RandomHorizontalFlip(p=0.5)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )
Dataset ImageFolder
    Number of datapoints: 3068
    Root location: D:\Dataset\RAF-DB\test
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=None)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )
['Anger', 'Disgust', 'Fear', 'Happiness', 'Neutral', 'Sadness', 'Surprise']
{'Anger': 0, 'Disgust': 1, 'Fear': 2, 'Happiness': 3, 'Neutral': 4, 'Sadness': 5, 'Surprise': 6}

['Anger', 'Disgust', 'Fear', 'Happiness', 'Neutral', 'Sadness', 'Surprise']
{'Anger':

In [8]:
#to(device)将模型加入GPU中加速计算
model = MobileNetV3_Large().to(device)
#设置优化器  
optimizer = optim.AdamW(model.parameters(), lr=lr)
#设置损失函数MobileNetV3_Large
criteon = nn.CrossEntropyLoss().to(device)
#余弦衰减学习率
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=80, eta_min=0,verbose = True)
#形如TensorFlow中的summary函数输出模型参数
summary(model, input_size=[(3, 224, 224)], batch_size=batch_size, device="cuda")
print()

#torch官方输出参数，目测不是很好
print("=================model summary ======================")
# print("VGG16 Net:", model)

Adjusting learning rate of group 0 to 5.0000e-04.
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1        [128, 16, 112, 112]             432
       BatchNorm2d-2        [128, 16, 112, 112]              32
            hswish-3        [128, 16, 112, 112]               0
            Conv2d-4        [128, 16, 112, 112]             256
       BatchNorm2d-5        [128, 16, 112, 112]              32
              ReLU-6        [128, 16, 112, 112]               0
            Conv2d-7        [128, 16, 112, 112]             144
       BatchNorm2d-8        [128, 16, 112, 112]              32
              ReLU-9        [128, 16, 112, 112]               0
           Conv2d-10        [128, 16, 112, 112]             256
      BatchNorm2d-11        [128, 16, 112, 112]              32
            Block-12        [128, 16, 112, 112]               0
           Conv2d-13        [128, 64, 112, 112]      

In [9]:
#测试函数
def evalute_(model,val_loader):
    model.eval()
    test_loss2 = 0.0
    test_corrects2 = 0.0
    number = 0
    for batchidx, (x, label) in enumerate(val_loader):
#         print(number)
    #torch.cuda.empty_cache()  #清除非必要GPU缓存，但是我建议不要在训练中使用此句，这可能会损失你相当多的时间
        number = number + 1
        x, label = x.to(device), label.to(device)
        #测试函数中加入no_grad()，如果不加会增加计算和显存
        with torch.no_grad():
            y1 = model(x)
            #虽然可以直接使用max函数，但是我建议在y1的比较重你最好使用F.softmax(y1,dim=1)，这样可能会有更好的效果，我在训练中使用了它
            _, preds1 = torch.max(F.softmax(y1,dim=1), 1)
            loss = criteon(y1, label)  
            
            test_loss2 += loss.item()*batch_size
            test_corrects2 += torch.sum(preds1 == label.data)
    #由于使用了最后一次抛弃，我不能使用全部测试集作为分母，这样会使最后的准确率变小
    test_loss1 = test_loss2 / (number*batch_size)
    test_acc1 = test_corrects2.double() / (number*batch_size)
#     print("TestDataset loss is ", test_loss1,"TestDataset accuracy is ",test_acc1)
    return test_acc1, test_loss1
print("执行结束")

执行结束


In [10]:
#关于AMP自动精度求解，我也并不是很熟悉，只能使用官方给的实例进行照葫芦画瓢。
scaler = torch.cuda.amp.GradScaler()
torch.cuda.empty_cache()
for epoch in range(D_epoch, epochs):
    time_one = time.time()                         #标记训练开始时间戳
    train_acc1 = 0.0
    train_loss1 = 0.0
    train_acc = 0.0
    train_loss = 0.0
    
    val_acc = 0.0
    val_loss = 0.0
    number = 0
    model.train()
    print("epoch:",epoch)

    for batchidx , (x ,label) in enumerate(data0_train):
        x , label = x.to(device), label.to(device)
        optimizer.zero_grad()
        with torch.cuda.amp.autocast():
            y1 = model(x)
            loss = criteon(y1,label)  
        _, preds1 = torch.max(F.softmax(y1,dim=1), 1)
        #AMP优化
        scaler.scale(loss).backward()
#         loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm = 5, norm_type=2)  #梯度剪裁
        scaler.step(optimizer)
#         optimizer.step()
        scaler.update()
        train_loss1 += loss.item()*batch_size
        train_acc1 += torch.sum(preds1 == label.data).double()
        number = number + 1
    time_two = time.time()             #标记训练结束时间戳
    #输出训练一轮所需时间，用于分析对比
    print("The Model-Train-Time spent  %d min %.2f s"%((time_two-time_one)//60,(time_two-time_one)%60))
    #计算训练时候的平均损失和平均准确率
    train_loss = train_loss1 / (number*batch_size)
    train_acc = train_acc1 / (number*batch_size)
    #计算测试时候的平均损失和平均准确率
    val_acc, val_loss = evalute_(model, data1_val)
    
    train_acc = train_acc.cpu()
    val_acc = val_acc.cpu()
    print('Accuracy : Train is {} , Valid is {} ;  Loss : Train is  {} ,Valid is {}'.format(train_acc, val_acc, train_loss , val_loss))
    #如果你不需要训练以及验证的准确率和损失值，你可以注释这下面的两行，它们不是非必须的，理论上只存在于汇报和论文中
    dataframe = pd.DataFrame(columns = [epoch,train_acc,train_loss,val_acc, val_loss])
    dataframe.to_csv(data_csv_path,line_terminator="\n",mode='a',index=False,sep=',')
    if val_acc > best_acc:
        print("覆盖最好的模型...")
        best_acc = val_acc 
        checkpoint = {
            'epoch': epoch,
            'model': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'best_acc':best_acc
        }
        torch.save(checkpoint,model_save_path)
#     time_three = time.time() 
#     print("测试花费时间",time_three-time_two)
    scheduler.step()  #动态学习率更新 
#如果你不是非必须，我建议你尽量不要使用n折交叉验证，使用数据增强可能效果更优于它。

epoch: 0
The Model-Train-Time spent  0 min 36.28 s
Accuracy : Train is 0.3748355263157895 , Valid is 0.382133152173913 ;  Loss : Train is  1.662230832953202 ,Valid is 1.6441715126452239
覆盖最好的模型...
Adjusting learning rate of group 0 to 4.9981e-04.
epoch: 1
The Model-Train-Time spent  0 min 32.54 s
Accuracy : Train is 0.4463815789473684 , Valid is 0.48335597826086957 ;  Loss : Train is  1.478145066060518 ,Valid is 1.521660918774812
覆盖最好的模型...
Adjusting learning rate of group 0 to 4.9923e-04.
epoch: 2
The Model-Train-Time spent  0 min 32.52 s
Accuracy : Train is 0.5305098684210526 , Valid is 0.5784646739130435 ;  Loss : Train is  1.2996336635790373 ,Valid is 1.1902399529581484
覆盖最好的模型...
Adjusting learning rate of group 0 to 4.9827e-04.
epoch: 3
The Model-Train-Time spent  0 min 31.85 s
Accuracy : Train is 0.5986842105263158 , Valid is 0.6188858695652174 ;  Loss : Train is  1.1117090658137674 ,Valid is 1.0806081994720127
覆盖最好的模型...
Adjusting learning rate of group 0 to 4.9692e-04.
epoch: 

The Model-Train-Time spent  0 min 31.92 s
Accuracy : Train is 0.9963815789473685 , Valid is 0.7503396739130435 ;  Loss : Train is  0.011565654734639745 ,Valid is 1.703002473582392
Adjusting learning rate of group 0 to 2.9877e-04.
epoch: 35
The Model-Train-Time spent  0 min 31.83 s
Accuracy : Train is 0.9952302631578948 , Valid is 0.75 ;  Loss : Train is  0.013010877968841478 ,Valid is 1.6823941210041875
Adjusting learning rate of group 0 to 2.8911e-04.
epoch: 36
The Model-Train-Time spent  0 min 32.52 s
Accuracy : Train is 0.9961348684210526 , Valid is 0.749320652173913 ;  Loss : Train is  0.011830421056794493 ,Valid is 1.766458537267602
Adjusting learning rate of group 0 to 2.7938e-04.
epoch: 37
The Model-Train-Time spent  0 min 31.94 s
Accuracy : Train is 0.9855263157894737 , Valid is 0.7398097826086957 ;  Loss : Train is  0.04288273174315691 ,Valid is 1.8536696434020996
Adjusting learning rate of group 0 to 2.6961e-04.
epoch: 38
The Model-Train-Time spent  0 min 31.91 s
Accuracy : T

The Model-Train-Time spent  0 min 33.31 s
Accuracy : Train is 1.0 , Valid is 0.764266304347826 ;  Loss : Train is  5.833629126611509e-05 ,Valid is 2.575608922087628
Adjusting learning rate of group 0 to 1.9030e-05.
epoch: 70
The Model-Train-Time spent  0 min 32.52 s
Accuracy : Train is 1.0 , Valid is 0.7734375 ;  Loss : Train is  9.81513018670835e-06 ,Valid is 2.5231981173805567
覆盖最好的模型...
Adjusting learning rate of group 0 to 1.5452e-05.
epoch: 71
The Model-Train-Time spent  0 min 32.61 s
Accuracy : Train is 1.0 , Valid is 0.7703804347826086 ;  Loss : Train is  4.039126399316286e-05 ,Valid is 2.6046608530956767
Adjusting learning rate of group 0 to 1.2236e-05.
epoch: 72
The Model-Train-Time spent  0 min 32.53 s
Accuracy : Train is 0.9998355263157894 , Valid is 0.7693614130434783 ;  Loss : Train is  0.0004246450862602184 ,Valid is 2.5477193023847495
Adjusting learning rate of group 0 to 9.3862e-06.
epoch: 73
The Model-Train-Time spent  0 min 32.65 s
Accuracy : Train is 1.0 , Valid is 0

The Model-Train-Time spent  0 min 32.68 s
Accuracy : Train is 1.0 , Valid is 0.7676630434782609 ;  Loss : Train is  7.20924060595663e-06 ,Valid is 3.001546989316526
Adjusting learning rate of group 0 to 1.1938e-04.
epoch: 106
The Model-Train-Time spent  0 min 32.76 s
Accuracy : Train is 0.9999177631578947 , Valid is 0.7615489130434783 ;  Loss : Train is  0.00022892481402346962 ,Valid is 3.0268928901008936
Adjusting learning rate of group 0 to 1.2784e-04.
epoch: 107
The Model-Train-Time spent  0 min 32.71 s
Accuracy : Train is 1.0 , Valid is 0.7646059782608695 ;  Loss : Train is  6.212960732610602e-06 ,Valid is 3.09780998851942
Adjusting learning rate of group 0 to 1.3650e-04.
epoch: 108
The Model-Train-Time spent  0 min 32.61 s
Accuracy : Train is 1.0 , Valid is 0.7703804347826086 ;  Loss : Train is  8.039794077998713e-06 ,Valid is 3.1269134283065796
Adjusting learning rate of group 0 to 1.4534e-04.
epoch: 109
The Model-Train-Time spent  0 min 32.62 s
Accuracy : Train is 1.0 , Valid is

The Model-Train-Time spent  0 min 31.80 s
Accuracy : Train is 0.9961348684210526 , Valid is 0.7615489130434783 ;  Loss : Train is  0.012185787733056043 ,Valid is 2.0754272263983022
Adjusting learning rate of group 0 to 4.3358e-04.
epoch: 141
The Model-Train-Time spent  0 min 31.86 s
Accuracy : Train is 0.9944078947368421 , Valid is 0.7452445652173912 ;  Loss : Train is  0.01767963873908708 ,Valid is 2.1878096746361773
Adjusting learning rate of group 0 to 4.4010e-04.
epoch: 142
The Model-Train-Time spent  0 min 31.86 s
Accuracy : Train is 0.9944901315789474 , Valid is 0.7510190217391304 ;  Loss : Train is  0.01727215717301557 ,Valid is 2.09852919889533
Adjusting learning rate of group 0 to 4.4633e-04.
epoch: 143
The Model-Train-Time spent  0 min 31.91 s
Accuracy : Train is 0.9942434210526315 , Valid is 0.7483016304347826 ;  Loss : Train is  0.017848801044257064 ,Valid is 2.2365041712056035
Adjusting learning rate of group 0 to 4.5225e-04.
epoch: 144
The Model-Train-Time spent  0 min 31

Accuracy : Train is 0.9725328947368421 , Valid is 0.735733695652174 ;  Loss : Train is  0.08479794928509939 ,Valid is 1.51983976882437
Adjusting learning rate of group 0 to 4.5787e-04.
epoch: 175
The Model-Train-Time spent  0 min 31.84 s
Accuracy : Train is 0.9754111842105263 , Valid is 0.7381114130434783 ;  Loss : Train is  0.0702830617561152 ,Valid is 1.4604970164920972
Adjusting learning rate of group 0 to 4.5225e-04.
epoch: 176
The Model-Train-Time spent  0 min 31.92 s
Accuracy : Train is 0.9794407894736842 , Valid is 0.7496603260869565 ;  Loss : Train is  0.05746123622122564 ,Valid is 1.3706237518269082
Adjusting learning rate of group 0 to 4.4633e-04.
epoch: 177
The Model-Train-Time spent  0 min 31.96 s
Accuracy : Train is 0.9819901315789473 , Valid is 0.7516983695652174 ;  Loss : Train is  0.052670058343363434 ,Valid is 1.603099885194198
Adjusting learning rate of group 0 to 4.4010e-04.
epoch: 178
The Model-Train-Time spent  0 min 31.86 s
Accuracy : Train is 0.9825657894736842 ,